In [8]:
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/e3/59/35a2892bf09ded9c1bf3804461efe772836a5261ef5dfb4e264ce813ff99/pandas-2.0.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 7.2 MB/s eta 0:00:0000:0100:01


In [6]:
!pip install numpy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/71/3c/3b1981c6a1986adc9ee7db760c0c34ea5b14ac3da9ecfcf1ea2a4ec6c398/numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 6.5 MB/s eta 0:00:0000:0100:01


In [10]:
!pip install seaborn

  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Obtaining dependency information for matplotlib!=3.6.1,>=3.1 from https://files.pythonhosted.org/packages/c2/da/a5622266952ab05dc3995d77689cba600e49ea9d6c51d469c077695cb719/matplotlib-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/aa/55/02c6d24804592b862b38a85c9b3283edc245081390a520ccd11697b6b24f/contourpy-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/87/61/f50ab3237b0cbf2b0be12274227f912d30f94e2b93fb8bae92c91107eee8/fonttools-4.42.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Using cached kiwisolver-1.4.4-c

In [11]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np
import pandas as pd
import seaborn as sns

In [145]:
def scrapping(genre):
    pages = [1,51,101,151,201]
    URL1 = f'https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&genres={genre}&languages=en&sort=user_rating,desc&start='
    URL2 = '&explore=genres&ref_=adv_nxt'
    headers = {'Accept-Language': 'en-US,en;q=0.8'} # If this is not specified, the default language is Mandarin
    
    #initialize empty lists to store the variables scraped
    genre_titles = []
    genre_imdb_ids = []
    genre_genres = []
    genre_imgdata = []
    genre_plot = []
    
    for page in pages:
      
       #get request for adventure
       response = get(URL1
                      + str(page)
                      + URL2, headers=headers)
      
       sleep(randint(8,15))
       
       #throw warning for status codes that are not 200
       if response.status_code != 200:
           warn('Request: {}; Status code: {}'.format(requests, response.status_code))
    
       #parse the content of current iteration of request
       page_html = BeautifulSoup(response.text, 'html.parser')
          
       movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
      
       #extract the 50 movies for that page
       for container in movie_containers:
    
            #title
            title = container.h3.a.text
            genre_titles.append(title)
    
            #imdb_id
            imdb_id = container.find('a')['href'].strip().split('/')[-2]
            genre_imdb_ids.append(imdb_id)
    
            #images
            img = f'http://img.omdbapi.com/?i={imdb_id}&h=600&apikey=7c8ba5e4'
            genre_imgdata.append(img)
                
            #genre
            genre_name = container.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',') # remove the whitespace character, strip, and split to create an array of genres
            genre_genres.append(genre_name)

            #plot
            plot_url = f'https://www.omdbapi.com/?i={imdb_id}&apikey=7c8ba5e4&plot=full'
            response = requests.get(plot_url)
            response.raise_for_status()  # raises exception when not a 2xx response
            if response.status_code != 204:
                response = response.json()
            genre_plot.append(response['Plot'])
    
    df = pd.DataFrame({'movie': genre_titles,
                          'imdb_id': genre_imdb_ids,
                          'genre': genre_genres,
                       'plot':genre_plot,
                        'image_url':genre_imgdata}
                              )
    return df

In [144]:
imdb_id = 'tt0482571'
plot_url = f'https://www.omdbapi.com/?i={imdb_id}&apikey=7c8ba5e4&plot=full'
response = requests.get(plot_url)
response.raise_for_status()  # raises exception when not a 2xx response
if response.status_code != 204:
    response = response.json()
response['Plot']

'In the end of the nineteenth century, in London, Robert Angier, his beloved wife Julia McCullough, and Alfred Borden are friends and assistants of a magician. When Julia accidentally dies during a performance, Robert blames Alfred for her death, and they become enemies. Both become famous and rival magicians, sabotaging the performance of the other on the stage. When Alfred performs a successful trick, Robert becomes obsessed trying to disclose the secret of his competitor with tragic consequences.'

In [146]:
mystery_df = scrapping("mystery")

In [147]:
mystery_df

,movie,imdb_id,genre,plot,image_url
0,Se7en,tt0114369,"[Crime, Drama, Mystery]",A film about two homicide detectives' (Morgan ...,http://img.omdbapi.com/?i=tt0114369&h=600&apik...
1,The Green Mile,tt0120689,"[Crime, Drama, Fantasy]","Death Row guards at a penitentiary, in the 193...",http://img.omdbapi.com/?i=tt0120689&h=600&apik...
2,The Prestige,tt0482571,"[Drama, Mystery, Sci-Fi]","In the end of the nineteenth century, in Londo...",http://img.omdbapi.com/?i=tt0482571&h=600&apik...
3,The Usual Suspects,tt0114814,"[Crime, Drama, Mystery]","Following a truck hijack in New York, five cri...",http://img.omdbapi.com/?i=tt0114814&h=600&apik...
4,Psycho,tt0054215,"[Horror, Mystery, Thriller]",Phoenix office worker Marion Crane is fed up w...,http://img.omdbapi.com/?i=tt0054215&h=600&apik...
...,...,...,...,...,...
245,Nothing But the Truth,tt1073241,"[Crime, Drama, Mystery]",Thinking Pulitzer Prize and hoping to bring do...,http://img.omdbapi.com/?i=tt1073241&h=600&apik...
246,The Wolf Man,tt0034398,"[Horror, Mystery, Romance]","Upon the death of his brother, Larry Talbot re...",http://img.omdbapi.com/?i=tt0034398&h=600&apik...
247,In the Valley of Elah,tt0478134,"[Crime, Drama, Mystery]",A retired military investigator works with a p...,http://img.omdbapi.com/?i=tt0478134&h=600&apik...
248,Talaash: The Answer Lies Within,tt1787988,"[Crime, Drama, Mystery]",Seeing reflections of Mumbai under the red lig...,http://img.omdbapi.com/?i=tt1787988&h=600&apik...


In [148]:
romance_df = scrapping("romance")

In [149]:
scifi_df = scrapping("scifi")

In [150]:
sport_df = scrapping("sport")

In [151]:
thriller_df = scrapping("thriller")

In [152]:
war_df = scrapping("war")

In [153]:
western_df = scrapping("western")

In [154]:
mystery_df.to_csv("mystery_df.csv")

In [155]:
romance_df.to_csv("romance_df.csv")

In [156]:
scifi_df.to_csv("scifi_df.csv")

In [157]:
sport_df.to_csv("sport_df.csv")

In [158]:
thriller_df.to_csv("thriller_df.csv")

In [159]:
war_df.to_csv("war_df.csv")

In [160]:
western_df.to_csv("western_df.csv")

In [177]:
western_df['plot'][6]

"When Senator Ransom Stoddard returns home to Shinbone for the funeral of Tom Doniphon, he recounts to a local newspaper editor the story behind it all. He had come to town many years before, a lawyer by profession. The stage was robbed on its way in by the local ruffian, Liberty Valance, and Stoddard has nothing to his name left save a few law books. He gets a job in the kitchen at the Ericson's restaurant and there meets his future wife, Hallie. The territory is vying for Statehood and Stoddard is selected as a representative over Valance, who continues terrorizing the town. When he destroys the local newspaper office and attacks the editor, Stoddard calls him out, though the conclusion is not quite as straightforward as legend would have it."

In [161]:
import requests
import os

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

def download_photos(genre, df):
    for index, row in df.iterrows():
        img_data = requests.get(row['image_url']).content
        imdb_id = row['imdb_id']
        with open(create_dir(f"{genre}") + "/" + f'{imdb_id}.jpg', 'wb') as handler:
            handler.write(img_data)

In [162]:
download_photos("mystery", mystery_df)

In [163]:
download_photos("romance", romance_df)

In [164]:
download_photos("scifi", scifi_df)

In [165]:
download_photos("sport", sport_df)

In [166]:
download_photos("thriller", thriller_df)

In [167]:
download_photos("war", war_df)

In [168]:
download_photos("western", western_df)

In [81]:
!git status

On branch webscraping_jess
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   webscraping_jess.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../.python-version
	.ipynb_checkpoints/
	webscraping.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [82]:
!git add webscraping.ipynb

In [83]:
!git commit -m "webscraping"

[webscraping_jess 1f7e039] webscraping
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 notebooks/webscraping.ipynb


In [84]:
!git push origin webscraping_jess

To github.com:chooj202/movie_genre_prediction.git
 ! [rejected]        webscraping_jess -> webscraping_jess (fetch first)
error: failed to push some refs to 'github.com:chooj202/movie_genre_prediction.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [86]:
!git pull origin master

remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15 (delta 6), reused 12 (delta 5), pack-reused 0
Unpacking objects: 100% (15/15), 3.32 KiB | 243.00 KiB/s, done.
From github.com:chooj202/movie_genre_prediction
 * branch            master     -> FETCH_HEAD
   00c39bb..08770b1  master     -> origin/master
error: Your local changes to the following files would be overwritten by merge:
	notebooks/webscraping_jess.ipynb
Please commit your changes or stash them before you merge.
Aborting


In [89]:
!git add webscraping_jess.ipynb